### Add model, connection and device to settings:

In [28]:
# библиотеки
import os
import sys
import inspect
import numpy as np
import re
import math
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from hybriddomain.envs.hs.model.model_main import ModelNet as Model
from hybriddomain.settings.settings_main import Settings
from ddeint import ddeint

# пути и названия файлов
modelFileName = 'problems/Diplom_Maslenikov'
conn_name = "connection.json"
device_conf_name = "devices.json"

#Ввод пароля для инициализации пользователя
model = Model()
model.io.loadFromFile(modelFileName)
settings = Settings(model, conn_name, device_conf_name, conn_name)

INFO:settings_main:device_conf_rpath
INFO:settings_main:devices.json


Please enter password for user imaslenikov:


 ·········


In [29]:
print("equations:")
print(model.equations)
print("params:")
print(model.params)
print("paramValues:")
print(model.paramValues)

equations:
[["U'= V", "V'= (b*sin(V(t-1))-V-k*e*U)/e"]]
params:
['e', 'k', 'b']
paramValues:
[{'e': 0.01, 'k': 2, 'b': 0.1}, {'e': 0.01, 'k': 2, 'b': 0.9}]


### Run solver:

In [30]:
# запуск процесса решения уравнения
from hybriddomain.solvers.hs.remoterun.progresses.progress_notebook import ProgressNotebook
from hybriddomain.solvers.hs.postproc.results.results_main import ResultPostprocNet as ResultPostproc
progress = ProgressNotebook(100, prefix='solving')

progress_params = ProgressNotebook(len(model.paramValues), prefix='param')
# display(progress_params.progress)

progress_solving = ProgressNotebook(100, prefix='solving')
# display(progress_solving.progress)
# progress_solving.succ(0)

result_postproc = ResultPostproc(modelFileName)

from hybriddomain.solvers.hs.remoterun.remoterun import remoteProjectRun

clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))

# parametrisation:
def run(event):
    model = Model()
    model.io.loadFromFile(modelFileName)
    
    for params_idx in range(len(model.paramValues)):
        
        progress_params.succ(params_idx)
        progress_solving.set_prefix("solving")
        
        # changing param:
        model.defaultParamsIndex = params_idx
        model.io.saveToFile(settings.paths["hd"]["json"])
        model = Model()
        model.io.loadFromFile(modelFileName)
        
        remoteProjectRun(settings, 1, progress_solving,remove_old=False)
        
        # result_postproc.extract_out(params_idx, results_params)
        result_postproc.rename_out(params_idx)
    display(Javascript("alert('Done');"))
    
bTest.on_click(run)

display(progress_params.progress)
display(progress_solving.progress)
 
display(bTest)

IntProgress(value=0, description='param: ', max=1)

IntProgress(value=0, description='solving: ', max=99)

Button(description='run Diplom_Maslenikov ', style=ButtonStyle())

### Results:

In [31]:
model.readResults(result_format=1)
# available result files:
print("result files: ", model.results_paths)
# available plot files:
print("plot files: ", model.plots_paths)

result files:  {'u': ['problems/Diplom_Maslenikov/out/u_seq0.out', 'problems/Diplom_Maslenikov/out/u_seq1.out']}
plot files:  {}


In [32]:
# объекте model хранится вся информация по решению задачи
# get results (for TRAC-11 result_format=1):
model.readResults(result_format=1)
model.results_paths
# available results:
print(model.results_arrays.keys())

dict_keys(['u'])


### Video:

In [33]:
# available plots:
model.plots_paths

{}

In [34]:
display(HTML(model.get_video('System',0)))
display(HTML(model.get_video('System',1)))
display(HTML(model.get_video('System',2)))
display(HTML(model.get_video('System',3)))
display(HTML(model.get_video('System',4)))

KeyError: 'System'

### U(x, t-fixed),V(x, t-fixed):

In [35]:
# отрисовывает результат
%matplotlib inline

#display(w)
time = model.results_arrays['u']['timevalues']
#print(time)

times = model.results_arrays['u']['timevalues']
#print(times)
result = model.results_arrays['u']['resvalues']
print(result[0][0][0][10])
print(len(times))
result_x = np.array([result[0][time][0][5] for time in times])
result_y = np.array([result[0][time][1][5] for time in times])

tt =np.linspace (0.0,1.0,101)
circle_x=np.array([10*math.cos(2*3.1415926*t) for t in tt])
circle_y=np.array([10*math.sin(2*3.1415926*t) for t in tt])


def f(x,t):
    result_x = np.array([result[x][time][0][t] for time in times])
    result_y = np.array([result[x][time][1][t] for time in times])
    # Сама траектория зависимость от t берется разное начало запуска
    plt.plot(result_x, result_y)
    # Круг на котором лежат начала траекторий 
    plt.plot(circle_x, circle_y)
w = interactive(f,x=(0,len(result)-1,1),t=(0,100,1))
display(w)
print(len(result))
plt.show()

8.09017
101


interactive(children=(IntSlider(value=0, description='x', max=1), IntSlider(value=50, description='t'), Output…

2


In [ ]:
### отрисовывает результат
%matplotlib inline

time = model.results_arrays['u']['timevalues']

times = model.results_arrays['u']['timevalues']
result = model.results_arrays['u']['resvalues']
#for time in times:
#    print(result[0][time][0][5])
#print(times)
result_x = np.array([result[4][time][0][5] for time in times])
result_y = np.array([result[4][time][1][5] for time in times])

radius = 10 
true_x = [radius*math.cos(4*poi) for poi in time]
true_y = [radius*math.sin(4*poi) for poi in time]
#синий действительное решение
plt.plot(true_x, true_y, lw=2)

#оранджевый решение нашего метода
plt.plot(result_x, result_y)


def model1(Y,t):
    x,y=Y(t)
    return np.array([4*y,-4*x])
g=lambda t : np.array ([10,0])

#количество точек для использования построения решения
tt =np.linspace (0.0,30.0,30000)
answer_DDE = ddeint (model1,g,tt)
#зеленое это то что рисует библиотечный метод
plt.plot(answer_DDE[:,0],answer_DDE[:,1])
plt.show()


In [ ]:
# отрисовывает отклонение по одной из координат с течением времени
%matplotlib inline

time = model.results_arrays['u']['timevalues']

times = model.results_arrays['u']['timevalues']
result = model.results_arrays['u']['resvalues']

result_x = np.array([result[4][time][0][0] for time in times])
result_y = np.array([result[4][time][1][0] for time in times])


radius = 10 
true_x = [radius*math.cos(4*poi) for poi in time]
true_y = [radius*math.sin(4*poi) for poi in time]
#синий действительное решение
plt.plot(time,true_x)

#оранджевый решение нашего метода
plt.plot(time,result_x)


def model1(Y,t):
    x,y=Y(t)
    return np.array([4*y,-4*x])
g=lambda t : np.array ([10,0])

#количество точек для использования построения решения
tt =np.linspace (0.0,30.0,30000)
answer_DDE = ddeint (model1,g,tt)
#зеленое это то что рисует библиотечный метод
plt.plot(tt,answer_DDE[:,0])
plt.show()




In [ ]:
# отрисовывает отклонение от точного решения
%matplotlib inline

time = model.results_arrays['u']['timevalues']

times = model.results_arrays['u']['timevalues']
result = model.results_arrays['u']['resvalues']

result_x = np.array([result[4][time][0][0] for time in times])
result_y = np.array([result[4][time][1][0] for time in times])


radius = 10 
true_x = [radius*math.cos(4*poi) for poi in time]
true_y = [radius*math.sin(4*poi) for poi in time]
#синий действительное решение
#plt.plot(time,true_x)

#оранджевый решение нашего метода
plt.plot(time,abs(true_x-result_x))


def model1(Y,t):
    x,y=Y(t)
    return np.array([4*y,-4*x])
g=lambda t : np.array ([10,0])

#количество точек для использования построения решения
tt =np.linspace (0.0,30.0,30000)
answer_DDE = ddeint (model1,g,tt)

i = 0
find_DDE = []
for poi in tt:
    b = abs(answer_DDE[i,0]-10*math.cos(4*poi))
    find_DDE.append(b)
    i+=1
plt.plot(tt, find_DDE, lw=2)


plt.show()
#погрешность верна именно на столько убегает решение DDE оранджевый цвет